optuna-dashboard 起動メモ\
!hostname -i\
!optuna-dashboard --host "172.17.0.2"  "sqlite:///MachineLearning/OptunaLogs/2022-10-23/optuna.sqlite3"\
\
↓optuna.sqlite3に入っているログの一覧確認

In [ ]:
!optuna studies --storage "sqlite:///./OptunaLogs/2022-10-24/optuna.sqlite3"

# データの読み込み

In [ ]:
!pwd

In [ ]:
#warningはpythonの標準ライブラリ．
#FutureWarnigが邪魔なので非表示にする．動作に支障が無ければ問題ない．また最適化によって解が収束しないときに出るConvergenceWarningも邪魔なので非表示にする．
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore", category=(FutureWarning, ConvergenceWarning))#対象のwarningsクラスはタプルで渡す必要があるらしい

In [ ]:
import pprint
import sys
sys.path.append("/mnt/MachineLearning")
pprint.pprint(sys.path)

In [ ]:
#各種モジュールのimport
%matplotlib inline
import os
import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mlxtend.plotting import scatterplotmatrix
from mlxtend.plotting import heatmap

from modules import show_mod
from modules.log_controler import ControlLog

from tqdm.notebook import tqdm

from sklearn import preprocessing
from sklearn.pipeline import make_pipeline

from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet

from sklearn.model_selection import cross_validate, KFold
from sklearn.metrics import mean_squared_error

import optuna

In [ ]:
#読み込むデータのパスの設定
current_dir_path = os.getcwd()
data_path = "/mnt/MachineLearning/data"
input_list = glob.glob(data_path+"/*"+"/inputdata.csv")
output_list = glob.glob(data_path+"/CountResults"+"/sitting"+"/count_from2sec_patientAverage.csv")
num_input = len(input_list)

In [ ]:
data_path

In [ ]:
#input(空調条件等)の読み込み
df_input = None

for input_fname in input_list:
    df_read = pd.read_csv(input_fname, index_col="case_name")
    if df_input is None:
        df_input = df_read
    else:
        df_input = df_input.append(df_read)

df_input

In [ ]:
#入力データの選択(オフィスサイズとエアコン位置が被っていると思うので削除)
delete_cols = ["aircon_position_x", "aircon_position_y"]
df_input.drop(df_input[delete_cols], axis=1, inplace=True)
df_input

In [ ]:
#排気口位置a,b,offをダミー変数化
df_input = pd.get_dummies(df_input, columns=['exhaust'])
df_input

In [ ]:
df_output = pd.read_csv(output_list[0], index_col="casename")
df_output

In [ ]:
#df_outputの値の幅を見てみる
#正規表現で書いており、.extractはパターンにマッチした最初の文字列のみを抜き出す。office+0-9までの数字1桁または2桁を探索
office_list = df_output.index.str.extract("(office+[0-9]{1,2})").to_numpy()

plt.figure(figsize=[10, 8])

#カラーマップ等の準備
markers = ("s", "x", "o", "^", "v", "<", ">", "1", "2", "3", "4", "8")
colors = ("red", "blue", "limegreen", "gray", "cyan", "black", "purple", "green",
          "orange", "yellow", "crimson", "goldenrod", "orchid")
start_index = 0

for idx, target_office_name in enumerate(np.unique(office_list)):
    target_office_index = [i for i in range(office_list.shape[0]) if any(office_list[i] == target_office_name)]
    plt.scatter(df_output.iloc[target_office_index ,2], range(1+start_index, len(target_office_index)+1+start_index), 
                s=80, c=colors[idx], marker=markers[2], edgecolor="white", label=target_office_name)
    start_index += len(target_office_index)
    

plt.ylabel("case number")
plt.xlabel("infected risk")
plt.legend(loc="best")
plt.tight_layout()
plt.grid()
plt.show()

In [ ]:
df_total = pd.merge(df_input, df_output, left_index=True, right_index=True)
print(f"num null in df:{df_total.isnull().values.sum()}")#結合が上手く行っていないかどうか確認
df_total

In [ ]:
df_total.info()

# 統計分析

In [ ]:
#基礎統計量について確認
df_total.describe()

In [ ]:
#口のz座標は全て同じであることがわかり特徴量として機能しないので削除
delete_cols = ["1_z","2_z","3_z","4_z","5_z"]
df_total.drop(df_total[delete_cols], axis=1, inplace=True)
df_total

In [ ]:
plot_cols = ["aircon", "ventilation", "office_size_x", "office_size_y", "exhaust_a", "exhaust_b", "exhaust_off", "RoI"]
# plot_cols = ['1_x','1_y','2_x','2_y','3_x','3_y','4_x','4_y','5_x','5_y',"RoI"]
# plot_cols = ['1_angle','2_angle','3_angle','4_angle','5_angle',"RoI"]
scatterplotmatrix(df_total[plot_cols].values, figsize=(20, 20), names=plot_cols, alpha=0.7)
plt.tight_layout()
plt.show()

外れ値の確認などもここで行っておくとよい．

In [ ]:
#ピアソンの積率相関係数のヒートマップ（ピアソンの積率相関係数はスケーリングした共分散）
cmap = show_mod.make_colormap(["blue", "lightgreen", "orange"])
correlationcoefficient = df_total[plot_cols].corr().to_numpy()
heat_map = heatmap(correlationcoefficient, figsize=(10, 8), cmap=cmap, row_names=plot_cols, column_names=plot_cols)
plt.show()

# データの前処理 

In [ ]:
#説明変数と目的変数の定義
explanatory_variable =['aircon', 'ventilation', '1_x', '1_y', '1_angle', '2_x', '2_y',
                       '2_angle', '3_x', '3_y', '3_angle', '4_x', '4_y', '4_angle', '5_x', '5_y', '5_angle',
                       'office_size_x','office_size_y', 'exhaust_a', 'exhaust_b', 'exhaust_off']
objective_variable = ["RoI"]

df_explanatory_variable = df_total[explanatory_variable]
df_objective_variable = df_total[objective_variable]

**標準化について**\
scikit-learnによる線型回帰では勾配降下法に基づく最適化を行わず，scipyの最小二乗法の実装に使用されているLAPACKに基づく高度な最適化手法を利用している．\
そのため使用する変数を標準化する必要がなく，寧ろ標準化しない方が上手く行くらしい．\
勾配降下法適用時に標準化する理由としては特徴量をスケーリングすることで，各特徴量に基づくパラメータ更新幅を揃えることができ，\
結果として誤差関数の収束が早くなったりする．\
標準化を行うべきかどうかは，用いる機械学習アルゴリズムによって異なるため，その都度考える必要がある．

In [ ]:
#説明変数の標準化(only explanatory variable)
#目的変数は標準化する意味がないため，するべきではない．
stdscaler = preprocessing.StandardScaler()
stdscaler.fit(df_explanatory_variable)
np_explanatory_variable_std = stdscaler.transform(df_explanatory_variable)
df_explanatory_variable_std = pd.DataFrame(np_explanatory_variable_std, index=df_explanatory_variable.index, columns=df_explanatory_variable.columns)
df_explanatory_variable_std

In [ ]:
df_explanatory_variable
# df_explanatory_variable_std
# df_objective_variable

**同じ形状のオフィス一覧**\
office1-6\
office7-9\
office10-15\
office16-21\
office22-27\
office28-33

In [ ]:
#特定のofficeだけテストデータにする
target_office_name = ["office10"]
#正規表現で書いており、.extractはパターンにマッチした最初の文字列のみを抜き出す。office+0-9までの数字1桁または2桁を探索
office_list = df_total.index.str.extract("(office+[0-9]{1,2})").to_numpy()
print(f"office list in data:\n{np.unique(office_list)}")
#リスト内包表記
test_data_index = [i for i in range(office_list.shape[0]) if any(office_list[i] == target_office_name)]
#test_data_index以外をtrain_data_indexとする
train_data_bool = np.ones(office_list.shape[0], dtype = bool)
train_data_bool[test_data_index] = False
train_data_index = np.arange(office_list.shape[0])[train_data_bool]

In [ ]:
#各オフィスで分けて統計解析してみる(同じ形状のオフィスだけで関係を見たりしたいときに使う)
# plot_cols = ["aircon", "ventilation", "office_size_x", "office_size_y", "exhaust_a", "exhaust_b", "exhaust_off", "RoI"]
plot_cols = ['1_x','1_y','2_x','2_y','3_x','3_y','4_x','4_y','5_x','5_y',"RoI"]
# plot_cols = ['1_angle','2_angle','3_angle','4_angle','5_angle',"RoI"]
df_target_office = df_total.iloc[test_data_index]
scatterplotmatrix(df_target_office[plot_cols].values, figsize=(20, 20), names=plot_cols, alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
#ピアソンの積率相関係数のヒートマップ（ピアソンの積率相関係数はスケーリングした共分散）
cmap = show_mod.make_colormap(["blue", "lightgreen", "orange"])
correlationcoefficient = df_total[plot_cols].corr().to_numpy()
heat_map = heatmap(correlationcoefficient, figsize=(10, 8), cmap=cmap, row_names=plot_cols, column_names=plot_cols)
plt.show()

# 回帰分析

# 線形回帰（線形重回帰）

今回は説明変数の数が一つではないため単回帰ではなく，重回帰となる．

In [ ]:
#使用するデータセットの決定（標準化するか否かなど）
#トレーニングデータ、テストデータの振り分け
train_explanatory_variable = df_explanatory_variable.iloc[train_data_index]
test_explanatory_variable = df_explanatory_variable.iloc[test_data_index]
train_objective_variable = df_objective_variable.iloc[train_data_index]
test_objective_variable = df_objective_variable.iloc[test_data_index]
test_explanatory_variable

In [ ]:
linear_model = LinearRegression()

In [ ]:
#k分割交差検証の実装
#評価指標の決定
metrics = ["r2", "neg_mean_squared_error", "neg_mean_absolute_error"]
#交差検証の分割方法を決定
kf = KFold(n_splits=10, shuffle=True, random_state=1)

scikit-learnによる交差検証では，よく"cross_val_score"が用いられるがこれは少し古く，\
現在は"cross_validate"というものがある．こちらではscoringにリストやタプル，辞書型を用いて複数の評価指標を与えることができる．\
まあfor文を使えばcross_val_scoreでも複数のscoringを使用することは簡単にできるけどね．

In [ ]:
scores = cross_validate(estimator=linear_model, scoring=metrics,
                         X=train_explanatory_variable, y=train_objective_variable, 
                         cv=kf, n_jobs=1, verbose=1, return_train_score=True)
pprint.pprint(scores)

n_jobsは使用するcpuの個数．これぐらいの軽い処理なら，分割せず1つでやった方が分割の処理に時間がかからないので逆に早い．

In [ ]:
print("交差検証結果の平均値")
print("test_r2 score:", scores["test_r2"].mean())
print("test_neg_mean_squared_error:", scores["test_neg_mean_squared_error"].mean())
print("test_neg_mean_absolute_error:", scores["test_neg_mean_absolute_error"].mean())

In [ ]:
linear_model.fit(train_explanatory_variable, train_objective_variable)
train_pred = linear_model.predict(train_explanatory_variable)
test_pred = linear_model.predict(test_explanatory_variable)
print("テストデータを用いた結果")
print(f"r2 train score:{linear_model.score(train_explanatory_variable, train_objective_variable)}")
print(f"MSE train score:{mean_squared_error(train_objective_variable, train_pred)}")
print(f"r2 test score:{linear_model.score(test_explanatory_variable, test_objective_variable)}")
print(f"MSE test score:{mean_squared_error(test_objective_variable, test_pred)}")

In [ ]:
#正規表現で書いており、.extractはパターンにマッチした最初の文字列のみを抜き出す。office+0-9までの数字1桁または2桁を探索
office_list = train_objective_variable.index.str.extract("(office+[0-9]{1,2})").to_numpy()

#残差プロット
show_mod.show_office_residual_plot(train_x=train_pred, train_y=train_pred-train_objective_variable.to_numpy(), 
                            test_x=test_pred, test_y=test_pred-test_objective_variable.to_numpy(), 
                            office_list=office_list)

## Lasso回帰

Lassoは正則化の特徴からパラメータのスケールによってペナルティを与えるので標準化必須

In [ ]:
#使用するデータセットの決定（標準化するか否かなど）
#トレーニングデータ、テストデータの振り分け
train_explanatory_variable = df_explanatory_variable_std.iloc[train_data_index]
test_explanatory_variable = df_explanatory_variable_std.iloc[test_data_index]
train_objective_variable = df_objective_variable.iloc[train_data_index]
test_objective_variable = df_objective_variable.iloc[test_data_index]
test_explanatory_variable
test_objective_variable

In [ ]:
#ラッソー回帰のインスタンス作成
lasso_model = Lasso(alpha=1.0)

In [ ]:
#k分割交差検証の実装
#評価指標の決定
metrics = ["r2", "neg_mean_squared_error", "neg_mean_absolute_error"]
#交差検証の分割方法を決定
kf = KFold(n_splits=10, shuffle=True, random_state=1)

In [ ]:
scores = cross_validate(estimator=lasso_model, scoring=metrics,
                         X=train_explanatory_variable, y=train_objective_variable, 
                         cv=kf, n_jobs=1, verbose=1, return_train_score=True)
pprint.pprint(scores)

In [ ]:
print("交差検証結果の平均値")
print("test_r2 score:", scores["test_r2"].mean())
print("test_neg_mean_squared_error:", scores["test_neg_mean_squared_error"].mean())
print("test_neg_mean_absolute_error:", scores["test_neg_mean_absolute_error"].mean())

In [ ]:
lasso_model.fit(train_explanatory_variable, train_objective_variable)
train_pred = lasso_model.predict(train_explanatory_variable)
test_pred = lasso_model.predict(test_explanatory_variable)
print("テストデータを用いた結果")
print(f"r2 train score:{lasso_model.score(train_explanatory_variable, train_objective_variable)}")
print(f"MSE train score:{mean_squared_error(train_objective_variable, train_pred)}")
print(f"r2 test score:{lasso_model.score(test_explanatory_variable, test_objective_variable)}")
print(f"MSE test score:{mean_squared_error(test_objective_variable, test_pred)}")

In [ ]:
#正規表現で書いており、.extractはパターンにマッチした最初の文字列のみを抜き出す。office+0-9までの数字1桁または2桁を探索
office_list = train_objective_variable.index.str.extract("(office+[0-9]{1,2})").to_numpy()

show_mod.show_office_residual_plot(train_x=train_pred, train_y=train_pred-train_objective_variable.to_numpy().reshape(-1), 
                            test_x=test_pred, test_y=test_pred-test_objective_variable.to_numpy().reshape(-1), 
                            office_list=office_list)

### optunaと交差検証を使用したハイパーパラメータ調整（自動最適化）

In [ ]:
class Objective:
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __call__(self, trial):
        #ハイパーパラメータの値設定
        params = {
            "alpha" : trial.suggest_loguniform("alpha", 0.0001, 10),
            "max_iter" : trial.suggest_int("max_iter", 100, 100000)
        }
        
        #**keywardargs(可変個数の変数をkeyと共に与えられる)
        model = Lasso(**params)
        
        #最適化実行時の評価指標の決定
        metrics = ["r2"]
        
        #交差検証の分割方法を決定
        kf = KFold(n_splits=10, shuffle=True, random_state=1)
        
        #評価指標の決定，k分割交差検証の実装
        scores = cross_validate(estimator=model, scoring=metrics,
                         X=self.X, y=self.y, 
                         cv=kf, n_jobs=1, verbose=0, return_train_score=True)
        
        return scores["test_r2"].mean()
        
        #複数の目的関数を戻り値とする
#         return scores["test_r2"].mean(), scores["test_neg_mean_squared_error"].mean()

In [ ]:
control_log = ControlLog()
sqlite_path = control_log.set_log()
study_name = control_log.decide_filename()
model_name = "Lasso"

#ハイパーパラメータの探索
objective = Objective(X=train_explanatory_variable, y=train_objective_variable)

#計算資源があるときはランダムサーチ，無ければTPESampler
#storageのパスにすでにDBファイルがあれば，それを読み込むのがload_if_exists
study = optuna.create_study(directions=["maximize"], study_name=model_name+"_"+study_name[0],
                            sampler=optuna.samplers.RandomSampler(), pruner=optuna.pruners.MedianPruner(),
                            storage=sqlite_path, load_if_exists=True)

"""
複数の目的関数を持つときはmulti_objectiveモジュールを使用する
複数の目的関数を最適化するのは多目的最適化と呼ばれ，専用の最適化アルゴリズムを指定する必要がある．単一目的関数の場合と比較する必要があるかもしれない．
directionは目的関数がどうなるように最適化したいか．samplerが多目的最適化アルゴリズム．defaultはNSGAIIMultiObjectiveSampler
"""

# study = optuna.multi_objective.create_study(directions=["maximize", "minimize"],
#                                             sampler=optuna.multi_objective.samplers.NSGAIIMultiObjectiveSampler(seed=1), 
#                                             storage=sqlite_path, load_if_exists=True)

#最適化の実行．n_trialsは何回実行するか．指定しなければできるだけやる．他にもtimeoutは計算にかける時間の上限値を秒単位で指定できる
#n_trialsまたはtimeoutのどちらかは指定したほうが良い．でないと永遠に計算し続け，pcが重くなる．
study.optimize(objective, n_trials=None, timeout=60, n_jobs=-1)

#ハンドラの削除．これを行わないとログファイルが上書きされる．
control_log.kill_handler()

In [ ]:
#探索の結果最も良かったscore
print(f"best score: {study.best_value}")
print(f"best params: {study.best_params}")

In [ ]:
#決定したハイパーパラメータを使用して全訓練データで学習，評価

best_model = Lasso(alpha=study.best_params["alpha"], max_iter=study.best_params["max_iter"])
best_model.fit(train_explanatory_variable, train_objective_variable)
train_pred = best_model.predict(train_explanatory_variable)
test_pred = best_model.predict(test_explanatory_variable)
print("テストデータを用いた結果")
print(f"r2 train score:{best_model.score(train_explanatory_variable, train_objective_variable)}")
print(f"MSE train score:{mean_squared_error(train_objective_variable, train_pred)}")
print(f"r2 test score:{best_model.score(test_explanatory_variable, test_objective_variable)}")
print(f"MSE test score:{mean_squared_error(test_objective_variable, test_pred)}")


#正規表現で書いており、.extractはパターンにマッチした最初の文字列のみを抜き出す。office+0-9までの数字1桁または2桁を探索
office_list = train_objective_variable.index.str.extract("(office+[0-9]{1,2})").to_numpy()
#予測結果の残差プロット
show_mod.show_office_residual_plot(train_x=train_pred, train_y=train_pred-train_objective_variable.to_numpy().reshape(-1), 
                            test_x=test_pred, test_y=test_pred-test_objective_variable.to_numpy().reshape(-1), 
                            office_list=office_list)

## Ridge回帰

In [ ]:
#使用するデータセットの決定（標準化するか否かなど）
#トレーニングデータ、テストデータの振り分け
train_explanatory_variable = df_explanatory_variable_std.iloc[train_data_index]
test_explanatory_variable = df_explanatory_variable_std.iloc[test_data_index]
train_objective_variable = df_objective_variable.iloc[train_data_index]
test_objective_variable = df_objective_variable.iloc[test_data_index]
test_explanatory_variable
test_objective_variable

In [ ]:
#リッジ回帰のインスタンス作成
ridge_model = Ridge(alpha=1.0)

In [ ]:
#k分割交差検証の実装
#評価指標の決定
metrics = ["r2", "neg_mean_squared_error", "neg_mean_absolute_error"]
#交差検証の分割方法を決定
kf = KFold(n_splits=10, shuffle=True, random_state=1)

In [ ]:
scores = cross_validate(estimator=ridge_model, scoring=metrics,
                         X=train_explanatory_variable, y=train_objective_variable, 
                         cv=kf, n_jobs=1, verbose=1, return_train_score=True)
pprint.pprint(scores)

In [ ]:
print("交差検証結果の平均値")
print("test_r2 score:", scores["test_r2"].mean())
print("test_neg_mean_squared_error:", scores["test_neg_mean_squared_error"].mean())
print("test_neg_mean_absolute_error:", scores["test_neg_mean_absolute_error"].mean())

In [ ]:
ridge_model.fit(train_explanatory_variable, train_objective_variable)
train_pred = ridge_model.predict(train_explanatory_variable)
test_pred = ridge_model.predict(test_explanatory_variable)
print("テストデータを用いた結果")
print(f"r2 train score:{lasso_model.score(train_explanatory_variable, train_objective_variable)}")
print(f"MSE train score:{mean_squared_error(train_objective_variable, train_pred)}")
print(f"r2 test score:{lasso_model.score(test_explanatory_variable, test_objective_variable)}")
print(f"MSE test score:{mean_squared_error(test_objective_variable, test_pred)}")

In [ ]:
#正規表現で書いており、.extractはパターンにマッチした最初の文字列のみを抜き出す。office+0-9までの数字1桁または2桁を探索
office_list = train_objective_variable.index.str.extract("(office+[0-9]{1,2})").to_numpy()

show_mod.show_office_residual_plot(train_x=train_pred, train_y=train_pred-train_objective_variable.to_numpy(), 
                            test_x=test_pred, test_y=test_pred-test_objective_variable.to_numpy(), 
                            office_list=office_list)

### optunaと交差検証を使用したハイパーパラメータ調整（自動最適化）

In [ ]:
class Objective:
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __call__(self, trial):
        #ハイパーパラメータの値設定
        params = {
            "alpha" : trial.suggest_loguniform("alpha", 0.0001, 10),
            "max_iter" : trial.suggest_int("max_iter", 100, 100000),
            "solver" : "auto",
            "positive" : False
        }
        
        #**keywardargs(可変個数の変数をkeyと共に与えられる)
        model = Ridge(**params)
        
        #最適化実行時の評価指標の決定
        metrics = ["r2"]
        
        #交差検証の分割方法を決定
        kf = KFold(n_splits=10, shuffle=True, random_state=1)
        
        #評価指標の決定，k分割交差検証の実装
        scores = cross_validate(estimator=model, scoring=metrics,
                         X=self.X, y=self.y, 
                         cv=kf, n_jobs=1, verbose=0, return_train_score=True)
        
        return scores["test_r2"].mean()

In [ ]:
control_log = ControlLog()
sqlite_path = control_log.set_log()
study_name = control_log.decide_filename()
model_name = "Ridge"

#ハイパーパラメータの探索
objective = Objective(X=train_explanatory_variable, y=train_objective_variable)

#計算資源があるときはランダムサーチ，無ければTPESampler
#storageのパスにすでにDBファイルがあれば，それを読み込むのがload_if_exists
study = optuna.create_study(directions=["maximize"], study_name=model_name+"_"+study_name[0],
                            sampler=optuna.samplers.RandomSampler(), pruner=optuna.pruners.MedianPruner(),
                            storage=sqlite_path, load_if_exists=True)

"""
複数の目的関数を持つときはmulti_objectiveモジュールを使用する
複数の目的関数を最適化するのは多目的最適化と呼ばれ，専用の最適化アルゴリズムを指定する必要がある．単一目的関数の場合と比較する必要があるかもしれない．
directionは目的関数がどうなるように最適化したいか．samplerが多目的最適化アルゴリズム．defaultはNSGAIIMultiObjectiveSampler
"""

# study = optuna.multi_objective.create_study(directions=["maximize", "minimize"],
#                                             sampler=optuna.multi_objective.samplers.NSGAIIMultiObjectiveSampler(seed=1), 
#                                             storage=sqlite_path, load_if_exists=True)

#最適化の実行．n_trialsは何回実行するか．指定しなければできるだけやる．他にもtimeoutは計算にかける時間の上限値を秒単位で指定できる
#n_trialsまたはtimeoutのどちらかは指定したほうが良い．でないと永遠に計算し続け，pcが重くなる．
study.optimize(objective, n_trials=None, timeout=60, n_jobs=-1)

#ハンドラの削除．これを行わないとログファイルが上書きされる．
control_log.kill_handler()

In [ ]:
#探索の結果最も良かったscore
print(f"best score: {study.best_value}")
print(f"best params: {study.best_params}")

In [ ]:
#決定したハイパーパラメータを使用して全訓練データで学習，評価

best_model = Ridge(alpha=study.best_params["alpha"], max_iter=study.best_params["max_iter"])
best_model.fit(train_explanatory_variable, train_objective_variable)
train_pred = best_model.predict(train_explanatory_variable)
test_pred = best_model.predict(test_explanatory_variable)
print("テストデータを用いた結果")
print(f"r2 train score:{best_model.score(train_explanatory_variable, train_objective_variable)}")
print(f"MSE train score:{mean_squared_error(train_objective_variable, train_pred)}")
print(f"r2 test score:{best_model.score(test_explanatory_variable, test_objective_variable)}")
print(f"MSE test score:{mean_squared_error(test_objective_variable, test_pred)}")


#正規表現で書いており、.extractはパターンにマッチした最初の文字列のみを抜き出す。office+0-9までの数字1桁または2桁を探索
office_list = train_objective_variable.index.str.extract("(office+[0-9]{1,2})").to_numpy()
#予測結果の残差プロット
show_mod.show_office_residual_plot(train_x=train_pred, train_y=train_pred-train_objective_variable.to_numpy(), 
                            test_x=test_pred, test_y=test_pred-test_objective_variable.to_numpy(), 
                            office_list=office_list)

## ElasticNet

In [ ]:
#エラスティックネットのインスタンス作成
elasticnet_model = ElasticNet(alpha=1.0, l1_ratio=0.5)）

In [ ]:
#k分割交差検証の実装
#評価指標の決定
metrics = ["r2", "neg_mean_squared_error", "neg_mean_absolute_error"]
#交差検証の分割方法を決定
kf = KFold(n_splits=10, shuffle=True, random_state=1)

In [ ]:
scores = cross_validate(estimator=elasticnet_model, scoring=metrics,
                         X=train_explanatory_variable, y=train_objective_variable, 
                         cv=kf, n_jobs=1, verbose=1, return_train_score=True)
pprint.pprint(scores)

In [ ]:
print("交差検証結果の平均値")
print("test_r2 score:", scores["test_r2"].mean())
print("test_neg_mean_squared_error:", scores["test_neg_mean_squared_error"].mean())
print("test_neg_mean_absolute_error:", scores["test_neg_mean_absolute_error"].mean())

In [ ]:
elasticnet_model.fit(train_explanatory_variable, train_objective_variable)
train_pred = elasticnet_model.predict(train_explanatory_variable)
test_pred = elasticnet_model.predict(test_explanatory_variable)

In [ ]:
#正規表現で書いており、.extractはパターンにマッチした最初の文字列のみを抜き出す。office+0-9までの数字1桁または2桁を探索
office_list = train_objective_variable.index.str.extract("(office+[0-9]{1,2})").to_numpy()

show_mod.show_office_residual_plot(train_x=train_pred, train_y=train_pred-train_objective_variable.to_numpy().reshape(-1), 
                            test_x=test_pred, test_y=test_pred-test_objective_variable.to_numpy().reshape(-1), 
                            office_list=office_list)

# 多項式回帰

## 多項式線形重回帰

In [ ]:
#使用するデータセットの決定（標準化するか否かなど）
#トレーニングデータ、テストデータの振り分け
train_explanatory_variable = df_explanatory_variable_std.iloc[train_data_index]
test_explanatory_variable = df_explanatory_variable_std.iloc[test_data_index]
train_objective_variable = df_objective_variable.iloc[train_data_index]
test_objective_variable = df_objective_variable.iloc[test_data_index]
train_explanatory_variable
# test_objective_variable

In [ ]:
#パイプラインの構築，attributeにアクセスできなくなったりするので使わない．やっぱ使う
def PolynomialRegression(n_components, degree, **kwargs):
    return make_pipeline(PCA(n_components=n_components), preprocessing.PolynomialFeatures(degree=degree), LinearRegression(**kwargs))

In [ ]:
#各主成分の分散説明率
pca = PCA(n_components=None)
pca.fit(train_explanatory_variable)
pca.explained_variance_ratio_

メモ：\
説明変数の数が22個もある．このまま多項式化しようとすると，項数が多すぎてメモリエラーとなり実行できない．（いけてdegree=5くらい）\
そのため説明変数を主成分分析によって減らした後，多項式回帰を実装してみようと思う．\
分散説明率の値からデータに大きく寄与しているのは大体11個位の成分なのでn_components=11でやってみる

In [ ]:
#多項式化したときの大きさを確認
degree = 10
poly_model = preprocessing.PolynomialFeatures(degree=degree)
train_explanatory_variable_pca_poly = poly_model.fit_transform(train_explanatory_variable_pca)
train_explanatory_variable_pca_poly.shape

In [ ]:
#多項式化したときの項を確認
poly_model.get_feature_names(train_explanatory_variable.columns)

In [ ]:
pca = PCA(n_components=11)
pca.fit(train_explanatory_variable)
train_explanatory_variable_pca = pd.DataFrame(pca.transform(train_explanatory_variable))

In [ ]:
class Objective:
    def __init__(self, X, y):
        #oputunaの最適化実行時に元データをそのまま参照しているとメモリの開放ができない
        self.X = X
        self.y = y
        
    def __call__(self, trial):
        #ハイパーパラメータの値設定
        n_components = trial.suggest_int("n_components", 1, 11)
        degree = trial.suggest_int("degree", 1, 5)#*argsにはiterableオブジェクトつまり，繰り返しができるtupleの形で与える必要がある
#         kwargs = {
#         }

        model = PolynomialRegression(n_components=n_components, degree=degree)
        
        #最適化実行時の評価指標の決定
        metrics = ["r2"]
        
        #交差検証の分割方法を決定
        kf = KFold(n_splits=10, shuffle=True, random_state=1)
        
        #評価指標の決定，k分割交差検証の実装
        scores = cross_validate(estimator=model, scoring=metrics,
                         X=self.X, y=self.y, 
                         cv=kf, n_jobs=1, verbose=0, return_train_score=False)
        
        return scores["test_r2"].mean()

In [ ]:
control_log = ControlLog()
sqlite_path = control_log.set_log()
study_name = control_log.decide_filename()
model_name = "Poly_Linear"

#ハイパーパラメータの探索
objective = Objective(X=train_explanatory_variable, y=train_objective_variable)

#計算資源があるときはランダムサーチ，無ければTPESampler
#storageのパスにすでにDBファイルがあれば，それを読み込むのがload_if_exists
study = optuna.create_study(directions=["maximize"], study_name=model_name+"_"+study_name[0],
                            sampler=optuna.samplers.RandomSampler(), pruner=optuna.pruners.MedianPruner(),
                            storage=sqlite_path, load_if_exists=True)

print(f"sutdy name: {study_name[0]}")

#最適化の実行．n_trialsは何回実行するか．指定しなければできるだけやる．他にもtimeoutは計算にかける時間の上限値を秒単位で指定できる
#n_trialsまたはtimeoutのどちらかは指定したほうが良い．でないと永遠に計算し続け，pcが重くなる．
study.optimize(objective, n_trials=None, timeout=120, n_jobs=-1)
# study.optimize(objective, n_trials=10000, timeout=None, n_jobs=5)

#ハンドラの削除．これを行わないとログファイルが上書きされる．
control_log.kill_handler()

In [ ]:
#探索の結果最も良かったscore
print("探索の結果最良のモデル")
print(f"best score: {study.best_value}")
print(f"best params: {study.best_params}")

In [ ]:
#決定したハイパーパラメータを使用して全訓練データで学習，評価

best_model = PolynomialRegression(n_components=study.best_params["n_components"], degree=study.best_params["degree"])
best_model.fit(train_explanatory_variable, train_objective_variable)
train_pred = best_model.predict(train_explanatory_variable)
test_pred = best_model.predict(test_explanatory_variable)
print("テストデータを用いた結果")
print(f"r2 train score:{best_model.score(train_explanatory_variable, train_objective_variable)}")
print(f"MSE train score:{mean_squared_error(train_objective_variable, train_pred)}")
print(f"r2 test score:{best_model.score(test_explanatory_variable, test_objective_variable)}")
print(f"MSE test score:{mean_squared_error(test_objective_variable, test_pred)}")


#正規表現で書いており、.extractはパターンにマッチした最初の文字列のみを抜き出す。office+0-9までの数字1桁または2桁を探索
office_list = train_objective_variable.index.str.extract("(office+[0-9]{1,2})").to_numpy()
#予測結果の残差プロット
show_mod.show_office_residual_plot(train_x=train_pred, train_y=train_pred-train_objective_variable.to_numpy(), 
                            test_x=test_pred, test_y=test_pred-test_objective_variable.to_numpy(), 
                            office_list=office_list)

超過学習．テストデータには一切適応できず．\
データ数を増やす，同じオフィス形状だけで試すなどすれば改善する可能性あり

## Lasso

In [ ]:
#使用するデータセットの決定（標準化するか否かなど）
#トレーニングデータ、テストデータの振り分け
train_explanatory_variable = df_explanatory_variable_std.iloc[train_data_index]
test_explanatory_variable = df_explanatory_variable_std.iloc[test_data_index]
train_objective_variable = df_objective_variable.iloc[train_data_index]
test_objective_variable = df_objective_variable.iloc[test_data_index]
test_explanatory_variable
test_objective_variable

In [ ]:
#パイプラインの構築
def LassoRegression(n_components, degree, **kwargs):
    return make_pipeline(PCA(n_components=n_components), preprocessing.PolynomialFeatures(degree=degree), Lasso(**kwargs))

In [ ]:
class Objective:
    
    def __init__(self, X, y, n_trials):
        #クラスにしとくとX,yをインスタンス変数にできるので，メモリを圧迫しないような気がする
        self.X = X
        self.y = y
        
        #tqdm関連の設定
        self.bar = tqdm(total = n_trials)
        self.bar.set_description('Progress rate')
        
    def __call__(self, trial):
        #ハイパーパラメータの値設定
        n_components = trial.suggest_int("n_components", 1, 11)
        degree = trial.suggest_int("degree", 1, 5)
        kwargs = {
            "alpha" : trial.suggest_loguniform("alpha", 0.0001, 10),
            "max_iter" : trial.suggest_int("max_iter", 1000, 100000)
        }
        
        #**keywardargs(可変個数の変数をkeyと共に与えられる)
        model = LassoRegression(n_components=n_components, degree=degree, **kwargs)
        
        #最適化実行時の評価指標の決定
        metrics = ["r2"]
        
        #交差検証の分割方法を決定
        kf = KFold(n_splits=10, shuffle=True, random_state=1)
        
        #評価指標の決定，k分割交差検証の実装
        scores = cross_validate(estimator=model, scoring=metrics,
                         X=self.X, y=self.y, 
                         cv=kf, n_jobs=1, verbose=0, return_train_score=False)
        
        self.bar.update(1)
        
        return scores["test_r2"].mean()

In [ ]:
control_log = ControlLog()
sqlite_path = control_log.set_log()
study_name = control_log.decide_filename()
model_name = "Poly_Lasso"

#訓練時のパラメータ設定
n_trials=200
timeout=None
n_jobs=-1

"""
最後のcontrol_log.kill_handler()が回らないとログが不必要に上書きされるので例外処理で最後まで必ず回るようにする．
exceptがtry内でエラーが生じたときの処理内容
finallyはtry内でエラーが生じたとき，生じなかったときどちらも動く処理
"""
try:
    #ハイパーパラメータの探索
    objective = Objective(X=train_explanatory_variable, y=train_objective_variable, n_trials=n_trials)

    #計算資源があるときはランダムサーチ，無ければTPESampler
    #storageのパスにすでにDBファイルがあれば，それを読み込むのがload_if_exists
    study = optuna.create_study(directions=["maximize"], study_name=model_name+"_"+study_name[0],
                                sampler=optuna.samplers.RandomSampler(), pruner=optuna.pruners.MedianPruner(),
                                storage=sqlite_path, load_if_exists=True)

    print(f"study name: {study_name[0]}")

    #最適化の実行．n_trialsは何回実行するか．指定しなければできるだけやる．他にもtimeoutは計算にかける時間の上限値を秒単位で指定できる
    #n_trialsまたはtimeoutのどちらかは指定したほうが良い．でないと永遠に計算し続け，pcが重くなる．
    study.optimize(objective, n_trials=n_trials, timeout=timeout, n_jobs=n_jobs)

except Exception as error:
    print(error)

finally:
    #ハンドラの削除．これを行わないとログファイルが上書きされる．
    control_log.kill_handler()

In [ ]:
#探索の結果最も良かったscoreとパラメータの表示
print("探索の結果最良のモデル")
print(f"best score: {study.best_value}")
print(f"best params: {study.best_params}")

In [ ]:
#決定したハイパーパラメータを使用して全訓練データで学習，評価

best_model = LassoRegression(n_components=study.best_params["n_components"], degree=study.best_params["degree"], 
                             alpha=study.best_params["alpha"], max_iter=study.best_params["max_iter"])
best_model.fit(train_explanatory_variable, train_objective_variable)
train_pred = best_model.predict(train_explanatory_variable)
test_pred = best_model.predict(test_explanatory_variable)
print("テストデータを用いた結果")
print(f"r2 train score:{best_model.score(train_explanatory_variable, train_objective_variable)}")
print(f"MSE train score:{mean_squared_error(train_objective_variable, train_pred)}")
print(f"r2 test score:{best_model.score(test_explanatory_variable, test_objective_variable)}")
print(f"MSE test score:{mean_squared_error(test_objective_variable, test_pred)}")


#正規表現で書いており、.extractはパターンにマッチした最初の文字列のみを抜き出す。office+0-9までの数字1桁または2桁を探索
office_list = train_objective_variable.index.str.extract("(office+[0-9]{1,2})").to_numpy()
#予測結果の残差プロット
show_mod.show_office_residual_plot(train_x=train_pred, train_y=train_pred-train_objective_variable.to_numpy().reshape(-1), 
                            test_x=test_pred, test_y=test_pred-test_objective_variable.to_numpy().reshape(-1), 
                            office_list=office_list)

トレーニングデータに関するフィッティングはかなり良くなっている\
しかしテストデータに関してはまだ厳しい

## Ridge

In [ ]:
#使用するデータセットの決定（標準化するか否かなど）
#トレーニングデータ、テストデータの振り分け
train_explanatory_variable = df_explanatory_variable_std.iloc[train_data_index]
test_explanatory_variable = df_explanatory_variable_std.iloc[test_data_index]
train_objective_variable = df_objective_variable.iloc[train_data_index]
test_objective_variable = df_objective_variable.iloc[test_data_index]
test_explanatory_variable
test_objective_variable

In [ ]:
#パイプラインの構築
def RidgeRegression(n_components, degree, **kwargs):
    return make_pipeline(PCA(n_components=n_components), preprocessing.PolynomialFeatures(degree=degree), Ridge(**kwargs))

In [ ]:
class Objective:
    
    def __init__(self, X, y, n_trials):
        self.X = X
        self.y = y
        
        #tqdm関連の設定
        self.bar = tqdm(total = n_trials)
        self.bar.set_description('Progress rate')
        
    def __call__(self, trial):
        #ハイパーパラメータの値設定
        n_components = trial.suggest_int("n_components", 1, 11)
        degree = trial.suggest_int("degree", 1, 5)
        kwargs = {
            "alpha" : trial.suggest_loguniform("alpha", 0.0001, 10),
            "max_iter" : trial.suggest_int("max_iter", 1000, 100000),
            "solver" : "auto",
            "positive" : False
        }
        
        #**keywardargs(可変個数の変数をkeyと共に与えられる)
        model = RidgeRegression(n_components=n_components, degree=degree, **kwargs)
        
        #最適化実行時の評価指標の決定
        metrics = ["r2"]
        
        #交差検証の分割方法を決定
        kf = KFold(n_splits=10, shuffle=True, random_state=1)
        
        #評価指標の決定，k分割交差検証の実装
        scores = cross_validate(estimator=model, scoring=metrics,
                         X=self.X, y=self.y, 
                         cv=kf, n_jobs=1, verbose=0, return_train_score=False)
        
        self.bar.update(1)
        
        return scores["test_r2"].mean()

In [ ]:
control_log = ControlLog()
sqlite_path = control_log.set_log()
study_name = control_log.decide_filename()
model_name = "Poly_Ridge"

#訓練時のパラメータ設定
n_trials=200
timeout=None
n_jobs=-1

"""
最後のcontrol_log.kill_handler()が回らないとログが不必要に上書きされるので例外処理で最後まで必ず回るようにする．
exceptがtry内でエラーが生じたときの処理内容
finallyはtry内でエラーが生じたとき，生じなかったときどちらも動く処理
"""
try:
    #ハイパーパラメータの探索
    objective = Objective(X=train_explanatory_variable, y=train_objective_variable, n_trials=n_trials)

    #計算資源があるときはランダムサーチ，無ければTPESampler
    #storageのパスにすでにDBファイルがあれば，それを読み込むのがload_if_exists
    study = optuna.create_study(directions=["maximize"], study_name=model_name+"_"+study_name[0],
                                sampler=optuna.samplers.RandomSampler(), pruner=optuna.pruners.MedianPruner(),
                                storage=sqlite_path, load_if_exists=True)

    print(f"study name: {study_name[0]}")

    #最適化の実行．n_trialsは何回実行するか．指定しなければできるだけやる．他にもtimeoutは計算にかける時間の上限値を秒単位で指定できる
    #n_trialsまたはtimeoutのどちらかは指定したほうが良い．でないと永遠に計算し続け，pcが重くなる．
    study.optimize(objective, n_trials=n_trials, timeout=timeout, n_jobs=n_jobs)

except Exception as error:
    print(error)

finally:
    #ハンドラの削除．これを行わないとログファイルが上書きされる．
    control_log.kill_handler()

In [ ]:
#探索の結果最も良かったscoreとパラメータの表示
print("探索の結果最良のモデル")
print(f"best score: {study.best_value}")
print(f"best params: {study.best_params}")

In [ ]:
#決定したハイパーパラメータを使用して全訓練データで学習，評価

best_model = RidgeRegression(n_components=study.best_params["n_components"], degree=study.best_params["degree"], 
                             alpha=study.best_params["alpha"], max_iter=study.best_params["max_iter"])
best_model.fit(train_explanatory_variable, train_objective_variable)
train_pred = best_model.predict(train_explanatory_variable)
test_pred = best_model.predict(test_explanatory_variable)
print("テストデータを用いた結果")
print(f"r2 train score:{best_model.score(train_explanatory_variable, train_objective_variable)}")
print(f"MSE train score:{mean_squared_error(train_objective_variable, train_pred)}")
print(f"r2 test score:{best_model.score(test_explanatory_variable, test_objective_variable)}")
print(f"MSE test score:{mean_squared_error(test_objective_variable, test_pred)}")


#正規表現で書いており、.extractはパターンにマッチした最初の文字列のみを抜き出す。office+0-9までの数字1桁または2桁を探索
office_list = train_objective_variable.index.str.extract("(office+[0-9]{1,2})").to_numpy()
#予測結果の残差プロット
show_mod.show_office_residual_plot(train_x=train_pred, train_y=train_pred-train_objective_variable.to_numpy(), 
                            test_x=test_pred, test_y=test_pred-test_objective_variable.to_numpy(), 
                            office_list=office_list)

## ElasticNet

In [ ]:
#使用するデータセットの決定（標準化するか否かなど）
#トレーニングデータ、テストデータの振り分け
train_explanatory_variable = df_explanatory_variable_std.iloc[train_data_index]
test_explanatory_variable = df_explanatory_variable_std.iloc[test_data_index]
train_objective_variable = df_objective_variable.iloc[train_data_index]
test_objective_variable = df_objective_variable.iloc[test_data_index]
test_explanatory_variable
test_objective_variable

In [ ]:
#パイプラインの構築
def ElasticNetRegression(n_components, degree, **kwargs):
    return make_pipeline(PCA(n_components=n_components), preprocessing.PolynomialFeatures(degree=degree), ElasticNet(**kwargs))

In [ ]:
class Objective:
    
    def __init__(self, X, y, n_trials):
        self.X = X
        self.y = y
        
        #tqdm関連の設定
        self.bar = tqdm(total = n_trials)
        self.bar.set_description('Progress rate')
        
    def __call__(self, trial):
        #ハイパーパラメータの値設定
        n_components = trial.suggest_int("n_components", 1, 11)
        degree = trial.suggest_int("degree", 1, 5)
        kwargs = {
            "l1_ratio" : trial.suggest_float("l1_ratio", 0, 1),
            "max_iter" : trial.suggest_int("max_iter", 1000, 100000),
        }
        
        #**keywardargs(可変個数の変数をkeyと共に与えられる)
        model = ElasticNetRegression(n_components=n_components, degree=degree, **kwargs)
        
        #最適化実行時の評価指標の決定
        metrics = ["r2"]
        
        #交差検証の分割方法を決定
        kf = KFold(n_splits=10, shuffle=True, random_state=1)
        
        #評価指標の決定，k分割交差検証の実装
        scores = cross_validate(estimator=model, scoring=metrics,
                         X=self.X, y=self.y, 
                         cv=kf, n_jobs=1, verbose=0, return_train_score=False)
        
        self.bar.update(1)
        
        return scores["test_r2"].mean()

In [ ]:
model_name = "Poly_ElasticNet"
#前回の続きから最適化を開始するかのスイッチ．Trueでリスタートする．
restart_switch = True

if restart_switch:
    #前回の続きから最適化を開始してみる(sutdy_nameが残っていないとできない．study_nameが残っていないときはoptunaログから自分で調査して与えればok)
#     study_name = 
    study = optuna.load_study(study_name=model_name+"_"+study_name[0], storage=sqlite_path)
    study.trials_dataframe()
    control_log = ControlLog()
    sqlite_path = control_log.set_log(*study_name)
    
else:
    control_log = ControlLog()
    sqlite_path = control_log.set_log()
    study_name = control_log.decide_filename()


#訓練時のパラメータ設定
n_trials=200
timeout=None
n_jobs=-1

"""
最後のcontrol_log.kill_handler()が回らないとログが不必要に上書きされるので例外処理で最後まで必ず回るようにする．
exceptがtry内でエラーが生じたときの処理内容
finallyはtry内でエラーが生じたとき，生じなかったときどちらも動く処理
"""
try:
    #ハイパーパラメータの探索
    objective = Objective(X=train_explanatory_variable, y=train_objective_variable, n_trials=n_trials)

    #計算資源があるときはランダムサーチ，無ければTPESampler
    #storageのパスにすでにDBファイルがあれば，それを読み込むのがload_if_exists
    study = optuna.create_study(directions=["maximize"], study_name=model_name+"_"+study_name[0],
                                sampler=optuna.samplers.RandomSampler(), pruner=optuna.pruners.MedianPruner(),
                                storage=sqlite_path, load_if_exists=True)

    print(f"study name: {study_name[0]}")

    #最適化の実行．n_trialsは何回実行するか．指定しなければできるだけやる．他にもtimeoutは計算にかける時間の上限値を秒単位で指定できる
    #n_trialsまたはtimeoutのどちらかは指定したほうが良い．でないと永遠に計算し続け，pcが重くなる．
    study.optimize(objective, n_trials=n_trials, timeout=timeout, n_jobs=n_jobs)

except Exception as error:
    print(error)

finally:
    #ハンドラの削除．これを行わないとログファイルが上書きされる．
    control_log.kill_handler()

In [ ]:
#探索の結果最も良かったscoreとパラメータの表示
print("探索の結果最良のモデル")
print(f"best score: {study.best_value}")
print(f"best params: {study.best_params}")

In [ ]:
#決定したハイパーパラメータを使用して全訓練データで学習，評価

best_model = ElasticNetRegression(n_components=study.best_params["n_components"], degree=study.best_params["degree"], 
                             alpha=study.best_params["l1_ratio"], max_iter=study.best_params["max_iter"])
best_model.fit(train_explanatory_variable, train_objective_variable)
train_pred = best_model.predict(train_explanatory_variable)
test_pred = best_model.predict(test_explanatory_variable)
print("テストデータを用いた結果")
print(f"r2 train score:{best_model.score(train_explanatory_variable, train_objective_variable)}")
print(f"MSE train score:{mean_squared_error(train_objective_variable, train_pred)}")
print(f"r2 test score:{best_model.score(test_explanatory_variable, test_objective_variable)}")
print(f"MSE test score:{mean_squared_error(test_objective_variable, test_pred)}")


#正規表現で書いており、.extractはパターンにマッチした最初の文字列のみを抜き出す。office+0-9までの数字1桁または2桁を探索
office_list = train_objective_variable.index.str.extract("(office+[0-9]{1,2})").to_numpy()
#予測結果の残差プロット
show_mod.show_office_residual_plot(train_x=train_pred, train_y=train_pred-train_objective_variable.to_numpy().reshape(-1), 
                            test_x=test_pred, test_y=test_pred-test_objective_variable.to_numpy().reshape(-1), 
                            office_list=office_list)

# サポートベクター回帰

### todo
* データ数とバイアス，バリアンスの関係を評価（データ数の問題がわかる可能性あり）
* 学習曲線の描画(optuna-dashboardを使用するとなるとsqliteの勉強が必要)
* MSE等の評価指標の取得は最終的な訓練後だけでいいかなと思っている（最適化中はいらないかな）
* 座標の対称移動などによるデータ数の増加はやってみる価値あり